<a href="https://colab.research.google.com/github/basketbob/ML/blob/main/netology_DS/web_scraping_KuzinVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```


#РЕШЕНИЕ

In [ ]:
def get_habr_articles(queries=[]):
  '''
  Ищет и парсит посты на сайте habr.com.
  На вход принимает список запросов для поиска.
  Возвращает датафрейм вида: <дата> - <заголовок> - <ссылка на материал>
  '''
  if type(queries) != list or len(queries) == 0:
    return 'Ошибка! Запрос не может быть пустым.'

  import requests
  import pandas as pd
  from bs4 import BeautifulSoup
  from datetime import datetime

  df = pd.DataFrame()

  #цикл по списку переданных запросов
  for query in queries:
    #посылаем запрос и формируем читабельный html
    r = requests.get('https://habr.com/ru/search/?q=python&target_type=posts')
    habr = BeautifulSoup(r.text, 'html.parser')
    #парсим список постов по запросу
    articles = habr.find_all('article', class_='tm-articles-list__item')

    #цикл по списку полученных постов
    for art in articles:
      row = {}
      row['id'] = art.get('id')
      row['title'] = art.find('h2').find('span').text
      row['link'] = art.find('a', class_='tm-article-snippet__title-link')['href']
      row['date'] = art.find('time').string
      #если пост опубликован в текущем году, то год не пишется, подставляем скриптом
      try:
        row['date'] = datetime.strptime(row['date'], "%d-%m-%y")
      except:
        row['date'] = row['date'].split()
        row['date'] = '-'.join([row['date'][0], row['date'][1], str(datetime.now().year)])

      df = pd.concat([df, pd.DataFrame([row])])

  return df

get_habr_articles(['sql', 'python']).head(20)


,id,title,link,date
0,646825,Курс «Python для инженеров». Старт 3 потока 31...,/ru/company/southbridge/news/t/646825/,20-января-2022
0,595093,Жаждущим автоматизации: открытый урок «ChatOps...,/ru/company/southbridge/news/t/595093/,13-декабря-2022
0,498364,"Вышел Python 2.7.18, последний релиз ветки Pyt...",/ru/news/t/498364/,21-апреля-2022
0,566370,Python Community Meetup 8/07: видео и материал...,/ru/company/raiffeisenbank/news/t/566370/,6-июля-2022
0,645485,Открытый урок «Пишем Custom Prometheus Exporte...,/ru/company/southbridge/news/t/645485/,13-января-2022
0,531402,Python как компилируемый статически типизирова...,/ru/news/t/531402/,4-декабря-2022
0,654707,Вышел мартовский релиз расширения Python для V...,/ru/news/t/654707/,8-марта-2022
0,490834,... начале этого года Python сместил Java и ...,/ru/company/itsumma/news/t/490834/,3-марта-2022
0,589555,EPAM разработала бесплатный курс по программир...,/ru/company/epam_systems/news/t/589555/,16-ноября-2022
0,505096,Вышла версия 0.0.2 snakeware — дистрибутива Li...,/ru/news/t/505096/,3-июня-2022
